<a href="https://colab.research.google.com/github/sugarforever/LangChain-Advanced/blob/main/Integrations/AutoGen/autogen_langchain_uniswap_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen + LangChain Use Case - Uniswap Protocol AI Agent

**`AutoGen`** is a versatile framework that facilitates the creation of LLM applications by employing multiple agents capable of interacting with one another to tackle tasks. These AutoGen agents can be tailored to specific needs, engage in conversations, and seamlessly integrate human participation. They are adaptable to different operation modes that encompass the utilization of LLMs, human inputs, and various tools.

**`LangChain`** is an open-source framework designed for software developers engaged in AI and ML. It enables them to seamlessly integrate LLM with external components, facilitating the creation of LLM-driven applications. The primary aim of LangChain is to establish connections between LLMs such as OpenAI's GPT-3.5 and GPT-4 and various external data sources, enabling the development and utilization of NLP applications.

Both of them are playing key roles in the LLM application development.

`AutoGen` doesn't support connecting to various external data sources natively. This is exactly where `LangChain` can come into play.

## Use Case - Uniswap Protocol

`Uniswap` is a decentralized exchange that allows users to trade Ethereum-based tokens.

In this tutorial, I will walk you through the steps to follow to build an AI agent that can complete tasks that involve Uniswap knowledge.

1. Build up a vector store with Uniswap V3 whitepaper.
2. Set up a conversational retrieval QA chain by LangChain, based on the vector store.
3. Define a function `answer_uniswap_question` by taking a parameter `question`, calling the QA chain to answer it.
4. Set up AutoGen user agent and assistant agent with function calling enabled.

  - In the function mapping, the function defined in step 3 is included.
  - The assistant agent is instructed by the following message to call the function `answer_uniswap_question` to answer Uniswap related questions.

    ```
    I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

    1. What is Uniswap?
    2. What are the main changes in Uniswap version 3?
    3. How to use Uniswap?

    Start the work now.
    ```

  - The function call is done by user agent.

### Environment Preparation

In [ ]:
%pip install pyautogen~=0.1.0 docker langchain openai tiktoken chromadb pypdf -q -U

In [1]:
!wget -O uniswap_v3.pdf https://uniswap.org/whitepaper-v3.pdf

--2023-12-10 17:03:47--  https://uniswap.org/whitepaper-v3.pdf
Resolving uniswap.org (uniswap.org)... 104.18.23.54, 104.18.22.54, 2606:4700::6812:1636, ...
Connecting to uniswap.org (uniswap.org)|104.18.23.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1500865 (1.4M) [application/pdf]
Saving to: ‘uniswap_v3.pdf’

uniswap_v3.pdf      100%[===================>]   1.43M  --.-KB/s    in 0.1s    

2023-12-10 17:03:47 (11.4 MB/s) - ‘uniswap_v3.pdf’ saved [1500865/1500865]



In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)
#
# Sample content of OAI_CONFIG_LIST file below:
#
# [
#   {
#     "model": "gpt-4",
#     "api_key": "your openai api key"
#   }
# ]
#

In [2]:
gpt4_api_key = config_list[0]["api_key"]

In [3]:
import os
os.environ['OPENAI_API_KEY'] = gpt4_api_key

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

### Steps

#### 1. Build up a vector store with Uniswap V3 whitepaper.

In [5]:
loaders = [ PyPDFLoader('./uniswap_v3.pdf') ]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [6]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()
)
vectorstore.add_documents(docs)

['abc68dda-9860-11ee-9a65-0242ac160002',
 'abc693fc-9860-11ee-9a65-0242ac160002',
 'abc6944c-9860-11ee-9a65-0242ac160002',
 'abc69488-9860-11ee-9a65-0242ac160002',
 'abc694c4-9860-11ee-9a65-0242ac160002',
 'abc6950a-9860-11ee-9a65-0242ac160002',
 'abc69550-9860-11ee-9a65-0242ac160002',
 'abc69596-9860-11ee-9a65-0242ac160002',
 'abc695d2-9860-11ee-9a65-0242ac160002',
 'abc69618-9860-11ee-9a65-0242ac160002',
 'abc69654-9860-11ee-9a65-0242ac160002',
 'abc69690-9860-11ee-9a65-0242ac160002',
 'abc696cc-9860-11ee-9a65-0242ac160002',
 'abc69708-9860-11ee-9a65-0242ac160002',
 'abc6974e-9860-11ee-9a65-0242ac160002',
 'abc6978a-9860-11ee-9a65-0242ac160002',
 'abc697c6-9860-11ee-9a65-0242ac160002',
 'abc6980c-9860-11ee-9a65-0242ac160002',
 'abc69848-9860-11ee-9a65-0242ac160002',
 'abc69884-9860-11ee-9a65-0242ac160002',
 'abc698ca-9860-11ee-9a65-0242ac160002',
 'abc69906-9860-11ee-9a65-0242ac160002',
 'abc69960-9860-11ee-9a65-0242ac160002',
 'abc6999c-9860-11ee-9a65-0242ac160002',
 'abc699d8-9860-

#### 2. Set up a conversational retrieval QA chain by LangChain, based on the vector store.

In [7]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [8]:
result = qa(({"question": "What is uniswap?"}))

In [9]:
result['answer']

' Uniswap is a noncustodial automated market maker implemented for the Ethereum Virtual Machine.'

#### 3. Define a function `answer_uniswap_question`

It takes a parameter `question`, calls the QA chain, and answer it by returning the answer from the chain response.

In [10]:
def answer_uniswap_question(question):
  response = qa({"question": question})
  return response["answer"]

#### 4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [14]:
llm_config={

    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_uniswap_question",
            "description": "Answer any Uniswap related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Uniswap protocol",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [15]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_uniswap_question": answer_uniswap_question}
)

### It's time to start a task for the agents.

Now, let's user the user agent to ask the agents to write an introduction blog for `Uniswap` protocol v3.

In [16]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.
"""
)

user_proxy (to assistant):


I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_uniswap_question *****
Arguments: 

{
  "question": "What is Uniswap?"
}
************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_uniswap_question...
user_proxy (to assistant):

***** Response from calling function "answer_uniswap_question" *****
 An automated market maker (AMM) is a type of algorithm that pools liquidity and makes it available to traders according to a predetermined set of rules. Uniswap v3 is an example of an AM